In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
)
select *
  from workspaces

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
)
select *
  from warehouse_scd

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
), query_history as (
     select date(q.start_time) as start_date,
            q.*,
            unix_timestamp(q.update_time) - unix_timestamp(q.end_time) as end_update_seconds,
            wh.warehouse_name,
            wh.workspace_name,
            wh.workspace_url
       from system.query.history q
       left join warehouse_scd wh
         on q.compute.warehouse_id = wh.warehouse_id
        and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
      where q.compute.warehouse_id is not null
        and start_time >= date_sub(current_timestamp(), :history_days)
)
select *
  from query_history
 

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
), query_history as (
     select date(q.start_time) as start_date,
            q.*,
            unix_timestamp(q.update_time) - unix_timestamp(q.end_time) as end_update_seconds,
            wh.warehouse_name,
            wh.workspace_name,
            wh.workspace_url
       from system.query.history q
       left join warehouse_scd wh
         on q.compute.warehouse_id = wh.warehouse_id
        and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
      where q.compute.warehouse_id is not null
        and start_time >= date_sub(current_timestamp(), :history_days)
)
select workspace_id,
       compute.warehouse_id,
       warehouse_name,
       start_time,
       client_application,
       client_driver,
       count(1)
  from query_history
 group by all

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
), query_history as (
     select date(q.start_time) as start_date,
            q.*,
            unix_timestamp(q.update_time) - unix_timestamp(q.end_time) as end_update_seconds,
            wh.warehouse_name,
            wh.workspace_name,
            wh.workspace_url
       from system.query.history q
       left join warehouse_scd wh
         on q.compute.warehouse_id = wh.warehouse_id
        and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
      where q.compute.warehouse_id is not null
        and start_time >= date_sub(current_timestamp(), :history_days)
)
select client_application,
       client_driver,
       start_date,
       max(total_duration_ms),
       min(total_duration_ms),
       max(end_update_seconds),
       min(end_update_seconds),
       count(1)
  from query_history
 where error_message is not null
   and error_message like 'Query has been timed out due to inactivity%'
 group by all

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
), query_history as (
     select date(q.start_time) as start_date,
            q.*,
            unix_timestamp(q.update_time) - unix_timestamp(q.end_time) as end_update_seconds,
            wh.warehouse_name,
            wh.workspace_name,
            wh.workspace_url
       from system.query.history q
       left join warehouse_scd wh
         on q.compute.warehouse_id = wh.warehouse_id
        and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
      where q.compute.warehouse_id is not null
        and start_time >= date_sub(current_timestamp(), :history_days)
)
select compute.warehouse_id,
       warehouse_name,
       workspace_id,
       start_date,
       substring(error_message, 1, 30) as error_message_prefix,
       count(1) as error_count
  from query_history
 where error_message is not null
   and not (startswith(error_message, '[TABLE_OR_VIEW_NOT_FOUND]') or
            startswith(error_message, '[INSUFFICIENT_PERMISSIONS]') or
            startswith(error_message, '[SCHEMA_NOT_FOUND]') or
            startswith(error_message, '[NO_SUCH_CATALOG_EXCEPTION]')
            )
 group by all

This query looks for dashboard queries that are repeats where they:
* Return 100,000 rows (should be cached)
* Does not use the query cache (`from_result_cache` = `false`)
* Run time of more than 10 minutes

This happens because we invalidate the query cache if the underlying data changes

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
), query_history as (
     select date(q.start_time) as start_date,
            q.*,
            unix_timestamp(q.update_time) - unix_timestamp(q.end_time) as end_update_seconds,
            wh.warehouse_name,
            wh.workspace_name,
            wh.workspace_url
       from system.query.history q
       left join warehouse_scd wh
         on q.compute.warehouse_id = wh.warehouse_id
        and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
      where q.compute.warehouse_id is not null
        and start_time >= date_sub(current_timestamp(), :history_days)
)
select statement_text,
       query_source.dashboard_id,
       workspace_url || '/dashboardsv3/' || query_source.dashboard_id || '/published' as dashboard_url,
       avg(total_duration_ms / 1000) as avg_duration_sec,
       count(1) as query_count,
       slice(collect_set(workspace_url || '/sql/history?queryId=' || statement_id), 1, 20) as statements
  from query_history
 where client_application = 'Databricks SQL Dashboard'
   and produced_rows < 100000
   and from_result_cache = false
   and statement_type = 'SELECT'
   and query_source.dashboard_id is not null
   and total_duration_ms > 600000 -- 10 mins
 group by all


Same query but this time we're looking for repeat queries that:
* read more than 1GB
* run for more than 10 mins

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
), query_history as (
     select date(q.start_time) as start_date,
            q.*,
            unix_timestamp(q.update_time) - unix_timestamp(q.end_time) as end_update_seconds,
            wh.warehouse_name,
            wh.workspace_name,
            wh.workspace_url
       from system.query.history q
       left join warehouse_scd wh
         on q.compute.warehouse_id = wh.warehouse_id
        and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
      where q.compute.warehouse_id is not null
        and start_time >= date_sub(current_timestamp(), :history_days)
)
select statement_text,
       query_source.dashboard_id,
       compute.warehouse_id,
       warehouse_name,
       workspace_url || '/dashboardsv3/' || query_source.dashboard_id || '/published' as dashboard_url,
       avg(total_duration_ms / 1000)          as avg_duration_sec,
       max(read_bytes / (1024 * 1024 * 1024)) as max_read_gb,
       min(read_bytes / (1024 * 1024 * 1024)) as min_read_gb,
       avg(read_bytes / (1024 * 1024 * 1024)) as avg_read_gb,
       count(1) as query_count,
       slice(collect_set(workspace_url || '/sql/history?queryId=' || statement_id), 1, 20) as statements
  from query_history
 where client_application = 'Databricks SQL Dashboard'
   and read_bytes >= (1024 * 1024 * 1024)
   and statement_type = 'SELECT'
   and query_source.dashboard_id is not null
   and total_duration_ms > 600000 -- 10 mins
 group by all

This query looks at number of queries by a given dashboard per day.
This is looking for dashboards that refresh often (eg every minute) and might not to.
This keeps a warehouse from shutting down and burning money as a result.

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
), query_history as (
     select date(q.start_time) as start_date,
            q.*,
            unix_timestamp(q.update_time) - unix_timestamp(q.end_time) as end_update_seconds,
            wh.warehouse_name,
            wh.workspace_name,
            wh.workspace_url
       from system.query.history q
       left join warehouse_scd wh
         on q.compute.warehouse_id = wh.warehouse_id
        and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
      where q.compute.warehouse_id is not null
        and start_time >= date_sub(current_timestamp(), :history_days)
)
select query_source.dashboard_id,
       workspace_url || '/dashboardsv3/' || query_source.dashboard_id || '/published' as dashboard_url,
       start_date,
       count(1) as query_count,
       slice(collect_set(workspace_url || '/sql/history?queryId=' || statement_id), 1, 20) as statements
  from query_history
 where client_application = 'Databricks SQL Dashboard'
   and statement_type = 'SELECT'
   and query_source.dashboard_id is not null
 group by all
 order by start_date desc,
          query_count desc

This one is dashboard counts of non-cached queries

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
), query_history as (
     select date(q.start_time) as start_date,
            q.*,
            unix_timestamp(q.update_time) - unix_timestamp(q.end_time) as end_update_seconds,
            wh.warehouse_name,
            wh.workspace_name,
            wh.workspace_url
       from system.query.history q
       left join warehouse_scd wh
         on q.compute.warehouse_id = wh.warehouse_id
        and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
      where q.compute.warehouse_id is not null
        and start_time >= date_sub(current_timestamp(), :history_days)
)
select query_source.dashboard_id,
       workspace_url || '/dashboardsv3/' || query_source.dashboard_id || '/published' as dashboard_url,
       count(1) as uncached_query_count,
       slice(collect_set(workspace_url || '/sql/history?queryId=' || statement_id), 1, 20) as statements
  from query_history
 where client_application = 'Databricks SQL Dashboard'
   and from_result_cache = false
   and statement_type = 'SELECT'
   and query_source.dashboard_id is not null
 group by all
